In [1]:
from _utils import *
from config import *

import time
import argparse
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import sparse
from numpy.linalg import inv as inv

import warnings
warnings.filterwarnings('ignore')

np.random.seed(28)

# 1 Functions

In [2]:
class args_config():
    def __init__(self, dataset_name):
        self.model_name = 'TRMF'
        self.model_result_save_path = os.path.join(result_folder_path, self.model_name)  
        create_folder(self.model_result_save_path)

        self.dataset_name = dataset_name
        self.dataset_result_save_path = os.path.join(self.model_result_save_path, self.dataset_name) 
        create_folder(self.dataset_result_save_path)
    
    def experiment_config(self, experiment_name):  
        self.experiment_result_save_path = os.path.join(self.dataset_result_save_path, experiment_name)
        create_folder(self.experiment_result_save_path)

        data_complete_path = os.path.join(self.experiment_result_save_path, 'data_complete')  
        best_data_complete_path = os.path.join(self.experiment_result_save_path, 'best_data_complete.csv')
        metrics_analysis_path = os.path.join(self.experiment_result_save_path, 'metrics_analysis')  
        plt_result_path = os.path.join(self.experiment_result_save_path, 'plt_result.png')
        best_args_path = os.path.join(self.experiment_result_save_path, 'best_args_config.yml')
        final_complete_save_path = os.path.join(self.experiment_result_save_path, 'final_complete.csv')
        create_folder([data_complete_path, metrics_analysis_path])
        
        parser = argparse.ArgumentParser(description='data complete configs')
        parser.add_argument('--data_complete_path', type=str, default=data_complete_path)
        parser.add_argument('--best_data_complete_path', type=str, default=best_data_complete_path)
        parser.add_argument('--metrics_analysis_path', type=str, default=metrics_analysis_path)
        parser.add_argument('--plt_result_path', type=str, default=plt_result_path)
        parser.add_argument('--best_args_path', type=str, default=best_args_path)
        parser.add_argument('--final_complete_save_path', type=str, default=final_complete_save_path)
        args = parser.parse_known_args()[0]
        return args

In [3]:
flow_sim_args = args_config('flow_sim')
flow_zcity_args = args_config('flow_zcity')
pres_sim_args = args_config('pres_sim')
pres_zcity_args = args_config('pres_zcity')

In [4]:
def print_result(it, tol, var, var_hat):
    print('Iter: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    num = len(var)
    mape = compute_mape(var, var_hat)
    rmse = compute_rmse(var, var_hat)
    smape = compute_smape(var, var_hat)
    print('complete Num: {}'.format(num))
    print('complete MAPE: {:.6}'.format(mape))
    print('complete RMSE: {:.6}'.format(rmse))
    print('complete SMAPE: {:.6}'.format(smape))
    return mape, rmse, smape

In [5]:
def metrics_analysis(mape_lists, rmse_lists, smape_lists, args, rank_search_space):
    mape_df = pd.DataFrame(mape_lists, index=rank_search_space, columns=['mape'])
    rmse_df = pd.DataFrame(rmse_lists, index=rank_search_space, columns=['rmse'])
    smape_df = pd.DataFrame(smape_lists, index=rank_search_space, columns=['smape'])
    mape_df.to_csv(os.path.join(args.metrics_analysis_path, 'mape_all_sensors.csv'), index=True, header=True)
    rmse_df.to_csv(os.path.join(args.metrics_analysis_path, 'rmse_all_sensors.csv'), index=True, header=True)
    smape_df.to_csv(os.path.join(args.metrics_analysis_path, 'smape_all_sensors.csv'), index=True, header=True)
    return

In [6]:
def compute_true_metrics(raw_data_value, denorm_sparse_value, denorm_data_complete_value):
    pos_test = np.where((raw_data_value != 0) & (denorm_sparse_value == 0))
    true_mape = compute_mape(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    true_rmse = compute_rmse(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    true_smape = compute_smape(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    return true_mape, true_rmse, true_smape

In [7]:
def fill_data_after_preprocess(best_data_complete_path, data_after_preprocess_path, data_after_complete_path):
    best_data_complete_df = read_csv_data(best_data_complete_path)
    data_after_preprocess_df = read_csv_data(data_after_preprocess_path)
   
    zero_indices = np.where(data_after_preprocess_df == 0)
    zero_indices = list(zip(zero_indices[0], zero_indices[1]))
    for zero_index in zero_indices:
        data_after_preprocess_df.iloc[zero_index[0], zero_index[1]] = best_data_complete_df.iloc[zero_index[0], zero_index[1]]
    data_after_preprocess_df.to_csv(data_after_complete_path, index=True, header=True)
    print('Data after complete saved in {}'.format(data_after_complete_path))
    return

In [8]:
def trmf(dense_mat, sparse_mat, init_para, init_hyper, time_lags, maxiter):
    """Temporal Regularized Matrix Factorization, TRMF."""
    
    ## Initialize parameters
    W = init_para["W"]
    X = init_para["X"]
    theta = init_para["theta"]
    
    ## Set hyperparameters
    lambda_w = init_hyper["lambda_w"]
    lambda_x = init_hyper["lambda_x"]
    lambda_theta = init_hyper["lambda_theta"]
    eta = init_hyper["eta"]
    
    dim1, dim2 = sparse_mat.shape
    pos_train = np.where(sparse_mat != 0)
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    binary_mat = sparse_mat.copy()
    binary_mat[pos_train] = 1
    d, rank = theta.shape
    
    for it in range(maxiter):
        ## Update spatial matrix W
        for i in range(dim1):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            vec0 = Xt.T @ sparse_mat[i, pos0[0]]
            mat0 = inv(Xt.T @ Xt + lambda_w * np.eye(rank))
            W[i, :] = mat0 @ vec0
        ## Update temporal matrix X
        for t in range(dim2):
            pos0 = np.where(sparse_mat[:, t] != 0)
            Wt = W[pos0[0], :]
            Mt = np.zeros((rank, rank))
            Nt = np.zeros(rank)
            if t < np.max(time_lags):
                Pt = np.zeros((rank, rank))
                Qt = np.zeros(rank)
            else:
                Pt = np.eye(rank)
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(np.where((t + time_lags >= np.max(time_lags)) & (t + time_lags < dim2)))[0]
                for k in index:
                    Ak = theta[k, :]
                    Mt += np.diag(Ak ** 2)
                    theta0 = theta.copy()
                    theta0[k, :] = 0
                    Nt += np.multiply(Ak, X[t + time_lags[k], :]
                                      - np.einsum('ij, ij -> j', theta0, X[t + time_lags[k] - time_lags, :]))
            vec0 = Wt.T @ sparse_mat[pos0[0], t] + lambda_x * Nt + lambda_x * Qt
            mat0 = inv(Wt.T @ Wt + lambda_x * Mt + lambda_x * Pt + lambda_x * eta * np.eye(rank))
            X[t, :] = mat0 @ vec0
        ## Update AR coefficients theta
        for k in range(d):
            theta0 = theta.copy()
            theta0[k, :] = 0
            mat0 = np.zeros((dim2 - np.max(time_lags), rank))
            for L in range(d):
                mat0 += X[np.max(time_lags) - time_lags[L] : dim2 - time_lags[L] , :] @ np.diag(theta0[L, :])
            VarPi = X[np.max(time_lags) : dim2, :] - mat0
            var1 = np.zeros((rank, rank))
            var2 = np.zeros(rank)
            for t in range(np.max(time_lags), dim2):
                B = X[t - time_lags[k], :]
                var1 += np.diag(np.multiply(B, B))
                var2 += np.diag(B) @ VarPi[t - np.max(time_lags), :]
            theta[k, :] = inv(var1 + lambda_theta * np.eye(rank) / lambda_x) @ var2

        mat_hat = W @ X.T
        mape = np.sum(np.abs(dense_mat[pos_test] - mat_hat[pos_test]) 
                      / dense_mat[pos_test]) / dense_mat[pos_test].shape[0]
        rmse = np.sqrt(np.sum((dense_mat[pos_test] - mat_hat[pos_test]) ** 2)/dense_mat[pos_test].shape[0])
        smape = compute_smape(dense_mat[pos_test], mat_hat[pos_test])
        
        if (it + 1) % 100 == 0:
            print('Iter: {}'.format(it + 1))
            print('Imputation MAPE: {:.6}'.format(mape))
            print('Imputation RMSE: {:.6}'.format(rmse))
            print('Imputation SMAPE: {:.6}'.format(smape))
            print()
    
    return mat_hat

In [9]:
class TRMF_dataComplete():

    def __init__(self, dataset_name, experiment_name, rank_search_space, experiment_args):
        if dataset_name == 'flow_sim':
            self.raw_dataset_path = flow_sim_path
        elif dataset_name == 'pres_sim':
            self.raw_dataset_path = pres_sim_path
        elif dataset_name == 'flow_zcity':
            self.raw_dataset_path = flow_zcity_path
        elif dataset_name == 'pres_zcity':
            self.raw_dataset_path = pres_zcity_path
        
        self.raw_data_df = read_csv_data(self.raw_dataset_path)
        self.rank_search_space = rank_search_space
        self.experiment_args = experiment_args

        self.data_norm_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, 'data_norm_for_completing.csv')
        self.sparse_mat_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, experiment_name, 'sparse_mat_for_completing.csv')
        self.denorm_sparse_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, experiment_name, 'denorm_sparse_for_completing.csv')

        self.data_complete_path = self.experiment_args.data_complete_path
        self.best_data_complete_path = self.experiment_args.best_data_complete_path
        self.metrics_analysis_path = self.experiment_args.metrics_analysis_path
        self.plt_result_path = self.experiment_args.plt_result_path
        self.best_args_path = self.experiment_args.best_args_path
        self.final_complete_save_path = self.experiment_args.final_complete_save_path        

        self._prepareForImputation()
        return
    
    def _prepareForImputation(self):
        self.norm_data_df = read_csv_data(self.data_norm_for_completing_path)
        self.dense_mat = self.norm_data_df.values
        self.dense_tensor = self.dense_mat.T.reshape([self.dense_mat.shape[1], -1, int(24*60/15)]).transpose(0, 2, 1)
        print('dense_tensor shape: ', self.dense_tensor.shape, ', it means (num_of_sensors, times_of_one_day, num_of_days)')
        print('dense_tensor中0的个数: ', np.sum(self.dense_mat == 0), ' 比例: %.4f' % (np.sum(self.dense_mat == 0) / np.prod(self.dense_mat.shape)))

        self.sparse_mat_df = read_csv_data(self.sparse_mat_for_completing_path)
        self.sparse_mat = self.sparse_mat_df.T.values
        self.sparse_tensor = self.sparse_mat.reshape([self.dense_mat.shape[1], -1, int(24*60/15)]).transpose(0, 2, 1)
        self.denorm_sparse_mat_df = read_csv_data(self.denorm_sparse_for_completing_path)
        self.dense_mat = self.dense_mat.T
        return

    def trmf_complete(self):

        mape_all_sensors_lists = []
        rmse_all_sensors_lists = []
        smape_all_sensors_lists = []
        mape_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        rmse_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        smape_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        best_rmse = np.inf
        best_rank = None
        best = False
        current_epoch = 1
        total_epoch = len(self.rank_search_space)
        for rank in self.rank_search_space:
            mape_all_sensors_list = []
            rmse_all_sensors_list = []
            smape_all_sensors_list = []

            start = time.time()
            dim1, dim2 = self.sparse_mat.shape
            time_lags = np.array([1, 2, 96])
            d = time_lags.shape[0]
            ## Initialize parameters
            W = 0.1 * np.random.rand(dim1, rank)
            X = 0.1 * np.random.rand(dim2, rank)
            theta = 0.1 * np.random.rand(d, rank)
            init_para = {"W": W, "X": X, "theta": theta}
            ## Set hyparameters
            lambda_w = 500
            lambda_x = 500
            lambda_theta = 500
            eta = 0.03
            init_hyper = {"lambda_w": lambda_w, "lambda_x": lambda_x, "lambda_theta": lambda_theta, "eta": eta}
            maxiter = 200
            print('epoch: %d/%d, rank: %d' % (current_epoch, total_epoch, rank))
        
            self.mat_hat = trmf(self.dense_mat, self.sparse_mat, init_para, init_hyper, time_lags, maxiter)
            self.data_complete_df = pd.DataFrame(self.mat_hat.T, index=self.norm_data_df.index, columns=self.norm_data_df.columns)
            self.denorm_data_complete_df = reverse_one2hundred_normalization(self.data_complete_df, self.raw_data_df)
            save_path = os.path.join(self.data_complete_path, 'rank=' + str(rank) + '.csv')
            
            self.denorm_data_complete_df = self.denorm_data_complete_df.round(3)
            self.denorm_data_complete_df.to_csv(save_path, index=True, header=True)

            mape_all_sensors, rmse_all_sensors, smape_all_sensors = compute_true_metrics(self.raw_data_df.values, self.denorm_sparse_mat_df.values, self.denorm_data_complete_df.values)
            print('MAPE: %.6f, RMSE: %.6f, SMAPE: %.6f' % (mape_all_sensors, rmse_all_sensors, smape_all_sensors))
            if rmse_all_sensors < best_rmse:
                best_rmse = rmse_all_sensors
                best_rank = rank
                best = True
            mape_all_sensors_list.append(mape_all_sensors)
            rmse_all_sensors_list.append(rmse_all_sensors)
            smape_all_sensors_list.append(smape_all_sensors)

            if best:
                self.denorm_data_complete_df.to_csv(self.best_data_complete_path, index=True, header=True)
                best = False

            mape_single_sensor_list, rmse_single_sensor_list, smape_single_sensor_list = result_analysis(self.raw_data_df, self.denorm_sparse_mat_df, self.denorm_data_complete_df)
            mape_single_sensor_df.loc['rank=' + str(rank), :] = mape_single_sensor_list
            rmse_single_sensor_df.loc['rank=' + str(rank), :] = rmse_single_sensor_list
            smape_single_sensor_df.loc['rank=' + str(rank), :] = smape_single_sensor_list
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            current_epoch += 1
            mape_all_sensors_lists.append(mape_all_sensors_list)
            rmse_all_sensors_lists.append(rmse_all_sensors_list)
            smape_all_sensors_lists.append(smape_all_sensors_list)
        mape_all_sensors_lists = np.array(mape_all_sensors_lists)
        rmse_all_sensors_lists = np.array(rmse_all_sensors_lists)
        smape_all_sensors_lists = np.array(smape_all_sensors_lists)

        metrics_analysis(mape_all_sensors_lists, rmse_all_sensors_lists, smape_all_sensors_lists, self.experiment_args, self.rank_search_space)
        mape_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'mape_single_sensor.csv'), index=True, header=True)
        rmse_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'rmse_single_sensor.csv'), index=True, header=True)
        smape_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'smape_single_sensor.csv'), index=True, header=True)
        print('最优参数为：rank = %.2f\n' % best_rank)
        best_mape_single_sensor_list = mape_single_sensor_df.loc['rank=' + str(rank), :].values
        best_rmse_single_sensor_list = rmse_single_sensor_df.loc['rank=' + str(rank), :].values
        best_smape_single_sensor_list = smape_single_sensor_df.loc['rank=' + str(rank), :].values
        return best_rank, best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list
    
    def run(self):
        print('\n--------------------------- Beginning ---------------------------')
        best_rank, best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list = self.trmf_complete()
        
        self.best_args_dict = {}
        self.best_args_dict['best_rank'] = best_rank
        write_yaml(self.best_args_path, self.best_args_dict)
        fill_data_after_preprocess(self.best_data_complete_path, self.raw_dataset_path, self.final_complete_save_path)
        plt_result(self.raw_data_df, read_csv_data(self.best_data_complete_path), self.denorm_sparse_mat_df, self.plt_result_path,
                   best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list)
        print('\n--------------------------- Ending ---------------------------')      


# 2 Experiments

------------

## 2.1 30% Random Missing

In [10]:
missing_type = 'random'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

## 2.2 60% Random Missing

In [ ]:
missing_type = 'random'
missing_rate = 0.6
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

------------

## 2.3 30% Long-Range Missing

In [ ]:
missing_type = 'long_range'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

-----

## 2.4 30% Block Missing

In [ ]:
missing_type = 'block'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

------

## 2.5 30% Mix Missing

In [ ]:
missing_type = 'mix'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

## 2.6 50% Mix Missing

In [ ]:
missing_type = 'mix'
missing_rate = 0.5
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
rank_search_space = [10, 20, 30]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = TRMF_dataComplete(dataset_name, experiment_name, rank_search_space, experiment_args)
Imputer.run()